In [1]:
from PIL import Image
from numpy import empty,array
from time import time

In [2]:
def prepare_text(text):
    lines = text.splitlines()   
    longest_string = len(max(lines, key = len))
    result = []
    for line in lines:
        result.append(list(line.ljust(longest_string,'\n')))    
    return result

## aho-corasick algorithm

In [3]:
class Node:
    def __init__(self, index):
        self.index = index
        self.output = []
        self.go_to = {}       
        self.failure = None    

        
def keyword_tree(patterns):
    root = Node(0)
    i = 1
    for pattern in patterns:
        node = root
        for letter in pattern:
            node = node.go_to.setdefault(letter, Node(i))
            i += 1
        node.output.append(pattern)
    return root


def build_matching_machine(patterns):
    root = keyword_tree(patterns)
    queue = []
    for node in root.go_to.values():
        queue.append(node)
        node.failure = root
    while len(queue) > 0:
        pop = queue.pop(0)
        for key, node in pop.go_to.items():
            queue.append(node)
            help_node = pop.failure
            while help_node != None and not key in help_node.go_to:
                help_node = help_node.failure
            node.failure = help_node.go_to[key] if help_node else root
            node.output += node.failure.output
    return root

In [4]:
def search_helper(text,pattern):
    res = set()
    K = []  
    t = -1
    K.append(t)
    for k in range(1, len(pattern) + 1):
        while(t >= 0 and pattern[t] != pattern[k - 1]):
            t = K[t]
        t = t + 1  
        K.append(t)
    m = 0  
    for i in range(0, len(text)):
        while (m >= 0 and pattern[m] != text[i]):
            m = K[m]
        m = m + 1  
        if m == len(pattern):
            res.add(i-m+1)
            m = K[m]
    return res

In [9]:
def matcher_2d_pattern(text,patterns,show_timer=True):
    start_time = time()
    aho_corasick = build_matching_machine(patterns)    
    
    patterns_pattern = []
    pattern_list = []
    num = 0
    for pattern in patterns:
        num += len(pattern)                
        patterns_pattern.append(num)
        pattern_list.append((pattern,num))        
    states = [[0 for _ in range(len(text[0]))] for _ in range(len(text))]

    for j in range(len(text[0])):
        node = aho_corasick
        for i in range(len(text)):
            while node != None and not text[i][j] in node.go_to:
                node = node.failure
            if node == None:
                node = aho_corasick
            else:
                node = node.go_to[text[i][j]]
                states[i][j] = node.index
    
    preparation_finish = time()
    pattern_matching = []
    i = -1 * len(patterns[0]) + 1
    count = 0
    for row in states:
        S = search_helper(row,patterns_pattern)       
        count += len(S)
        for j in S:
            pattern_matching.append(str(((i, j), (i, j+len(patterns)-1))))
            pattern_matching.append(str(((i+len(patterns[0])-1, j), (i+len(patterns[0])-1, j+len(patterns)-1))))               
        i += 1
    algo_finish = time()
    if show_timer:
        print("Automata preparation: ", str((preparation_finish-start_time)))
        print("Algorithm time: ", str((algo_finish-preparation_finish)))
    return count, pattern_matching

## Pattern search in text

In [10]:
haystack_text = prepare_text(open("haystack.txt").read())

In [11]:
patterns = []
for line in haystack_text:
    for letter in line:
        if letter not in patterns and letter != "\n":
            patterns += letter

In [12]:
for letter in sorted(patterns):
    print(letter, "->", matcher_2d_pattern(haystack_text,[letter+letter],False)[0])

  -> 158
" -> 0
( -> 0
) -> 0
, -> 0
- -> 0
. -> 0
0 -> 0
1 -> 0
3 -> 0
5 -> 0
6 -> 0
7 -> 0
: -> 0
; -> 0
A -> 0
B -> 0
C -> 0
D -> 0
E -> 0
F -> 0
G -> 0
I -> 0
L -> 0
M -> 0
N -> 0
O -> 0
P -> 0
Q -> 0
R -> 0
S -> 0
T -> 0
U -> 0
V -> 0
W -> 0
X -> 0
a -> 28
b -> 0
c -> 6
d -> 1
e -> 48
f -> 2
g -> 0
h -> 4
i -> 13
j -> 0
k -> 0
l -> 5
m -> 5
n -> 15
o -> 21
p -> 2
q -> 0
r -> 21
s -> 19
t -> 41
u -> 0
v -> 0
w -> 2
x -> 1
y -> 1
z -> 0


In [13]:
print("'th' ->", matcher_2d_pattern(haystack_text,['tt','hh'], False)[0])

'th' -> 0


In [14]:
print("'t h' ->",matcher_2d_pattern(haystack_text,['tt','  ','hh'], False)[0])
matcher_2d_pattern(haystack_text,['tt','  ','hh'], False)[1]

't h' -> 1


['((37, 0), (37, 2))', '((38, 0), (38, 2))']

## Image

In [15]:
def convert_image_to_1_0_matrix(image):
    return array(image.convert('P', palette=Image.ADAPTIVE, colors=256))

In [16]:
haystack_image = Image.open('haystack.png')
haystack_image_data = convert_image_to_1_0_matrix(haystack_image)

In [17]:
def get_image_chunk(left,top,right,bottom):
    data = []
    for row in haystack_image_data[top:bottom]:
        data.append(row[left:right])
    return data

In [18]:
def get_columns_from_pattern(pattern):
    columns = []
    for j in range(len(pattern[0])):
        column = []
        for i in range(len(pattern)):
            column.append(pattern[i][j])
        columns.append(column)
    return columns

In [19]:
image_e = haystack_image.crop((590,565,600,576))
image_e_pattern = get_image_chunk(590,565,600,576)
image_e

In [20]:
matcher_2d_pattern(haystack_image_data,get_columns_from_pattern(image_e_pattern),False)[0]

576

In [21]:
image_c = haystack_image.crop((486,564,494,575))
image_c_pattern = get_image_chunk(486,564,494,575)
image_c

In [22]:
matcher_2d_pattern(haystack_image_data,get_columns_from_pattern(image_c_pattern),False)[0]

213

In [23]:
image_o = haystack_image.crop((214,564,223,576))
image_o_pattern = get_image_chunk(214,564,223,576)
image_o

In [24]:
matcher_2d_pattern(haystack_image_data,get_columns_from_pattern(image_o_pattern),False)[0]

0

In [25]:
image_a = haystack_image.crop((120,234,129,245))
image_a_pattern = get_image_chunk(120,234,129,245)
image_a

In [26]:
matcher_2d_pattern(haystack_image_data,get_columns_from_pattern(image_a_pattern),False)[0]

397

In [27]:
image_pattern = haystack_image.crop((422,494,530,515))
image_pattern_pattern = get_image_chunk(422,494,530,515)
image_pattern

In [28]:
matcher_2d_pattern(haystack_image_data,get_columns_from_pattern(image_pattern_pattern), False)[0]

0

### Compare time for building automata and pattern matching

In [29]:
test_pattern_1 = ['abc','def','123']
matcher_2d_pattern(haystack_text,test_pattern_1)[0]

Automata preparation:  0.0029981136322021484
Algorithm time:  0.0019998550415039062


0

In [30]:
test_pattern_2 = ['and']
matcher_2d_pattern(haystack_text,test_pattern_2)

Automata preparation:  0.0019996166229248047
Algorithm time:  0.0019996166229248047


(1, ['((54, 52), (54, 52))', '((56, 52), (56, 52))'])

In [31]:
test_pattern_3 = ['aaaaaaaaaaaaaaaaaaaaaa']
matcher_2d_pattern(haystack_text,test_pattern_3)

Automata preparation:  0.0019991397857666016
Algorithm time:  0.0019991397857666016


(0, [])

In [32]:
test_pattern_3 = ['of', 'and', 'the']
matcher_2d_pattern(haystack_text,test_pattern_3)

Automata preparation:  0.0029973983764648438
Algorithm time:  0.001999378204345703


(0, [])

In [33]:
test_pattern_4 = ['zzzzzzzzzzzzzzzzaaaaaaaaaaaaafffffffff']
matcher_2d_pattern(haystack_text,test_pattern_3)

Automata preparation:  0.0029985904693603516
Algorithm time:  0.0019986629486083984


(0, [])

In [36]:
matcher_2d_pattern(haystack_image_data,get_columns_from_pattern(image_pattern_pattern))[0]

Automata preparation:  2.4136221408843994
Algorithm time:  0.4827232360839844


0

In [37]:
matcher_2d_pattern(haystack_image_data,get_columns_from_pattern(image_a_pattern))[0]

Automata preparation:  2.2057406902313232
Algorithm time:  0.3927750587463379


397